In [7]:
import os
import random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import cv2  # OpenCV for image processing

# =============================================================================
# 1. PATHS AND DATA SETUP
# =============================================================================
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
data_path = project_root / "data"
train_path = data_path / "train/images"
labels_path = data_path / "train/labels"

# Collect all image and annotation files
image_files = list(train_path.glob("*.jpg")) + list(train_path.glob("*.png"))
annotation_files = list(labels_path.glob("*.txt"))

# Class names
class_names = ['crab', 'fish', 'jellyfish', 'shrimp', 'small_fish', 'starfish']

# =============================================================================
# 2. YOLO ANNOTATION PARSING
# =============================================================================
def parse_yolo_annotations(annotation_path, image_width, image_height):
    """
    Parses a YOLO-style annotation file and converts normalized bbox coordinates
    into pixel-based bounding boxes (xmin, ymin, xmax, ymax).
    """
    bboxes = []
    with open(annotation_path, "r") as file:
        for line in file.readlines():
            data = line.strip().split()
            class_id = int(data[0])
            x_center, y_center, width, height = map(float, data[1:])

            xmin = int((x_center - width / 2) * image_width)
            ymin = int((y_center - height / 2) * image_height)
            xmax = int((x_center + width / 2) * image_width)
            ymax = int((y_center + height / 2) * image_height)

            bboxes.append({
                "name": class_names[class_id],
                "xmin": xmin,
                "ymin": ymin,
                "xmax": xmax,
                "ymax": ymax
            })
    return bboxes

# =============================================================================
# 3. DRAW BOUNDING BOXES (ON PIL IMAGES)
# =============================================================================
def draw_bounding_boxes(pil_image, bboxes, color="red"):
    """
    Draws bounding boxes and labels on a copy of the input PIL image.
    Returns a new PIL Image with bounding boxes.
    """
    draw_image = pil_image.copy()
    draw = ImageDraw.Draw(draw_image)

    # Attempt to load a TTF font; if unavailable, fall back to default
    try:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    for bbox in bboxes:
        # Draw rectangle
        draw.rectangle(
            [(bbox["xmin"], bbox["ymin"]), (bbox["xmax"], bbox["ymax"])],
            outline=color,
            width=3
        )
        # Draw label
        text_position = (bbox["xmin"], max(0, bbox["ymin"] - 20))
        draw.text(text_position, bbox["name"], fill=color, font=font)

    return draw_image

# =============================================================================
# 4. PREPROCESSING METHODS FOR UNDERWATER IMAGES
# =============================================================================

def white_balance_bgr(cv_bgr):
    """
    Applies a simple "gray-world" white balance to a BGR image (OpenCV format).
    Returns the corrected BGR image.
    """
    # Convert to float for precision
    result = cv_bgr.astype(np.float32) / 255.0

    # Compute mean of each channel
    avgB = np.mean(result[:,:,0])
    avgG = np.mean(result[:,:,1])
    avgR = np.mean(result[:,:,2])

    # Gray-world assumption
    grayValue = (avgB + avgG + avgR) / 3
    result[:,:,0] *= (grayValue / (avgB + 1e-8))
    result[:,:,1] *= (grayValue / (avgG + 1e-8))
    result[:,:,2] *= (grayValue / (avgR + 1e-8))

    # Clip to [0, 1], then scale back to [0, 255]
    result = np.clip(result, 0, 1)
    result = (result * 255).astype(np.uint8)
    return result

def enhance_clahe_bgr(cv_bgr, clip_limit=2.0, grid_size=(8, 8)):
    """
    Enhances a BGR image (OpenCV) using CLAHE in LAB color space
    to improve contrast in underwater images.
    """
    # Convert to LAB color space
    lab = cv2.cvtColor(cv_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE on the L-channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
    cl = clahe.apply(l)

    # Merge and convert back to BGR
    limg = cv2.merge((cl, a, b))
    enhanced_bgr = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return enhanced_bgr

def gamma_correction_bgr(cv_bgr, gamma=1.2):
    """
    Applies gamma correction to a BGR image (OpenCV).
    Gamma > 1.0 brightens the image, while Gamma < 1.0 darkens it.
    """
    # Build a lookup table for gamma values [0..255]
    invGamma = 1.0 / gamma
    table = np.array([
        ((i / 255.0) ** invGamma) * 255
        for i in np.arange(256)
    ]).astype("uint8")

    # Apply gamma correction using the lookup table
    return cv2.LUT(cv_bgr, table)

def pil_to_bgr(pil_img):
    """
    Converts a PIL (RGB) image into OpenCV BGR format.
    """
    cv_image = np.array(pil_img)
    cv_image = cv_image[:, :, ::-1].copy()  # RGB -> BGR
    return cv_image

def bgr_to_pil(cv_bgr):
    """
    Converts an OpenCV BGR image into a PIL (RGB) image.
    """
    cv_rgb = cv2.cvtColor(cv_bgr, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(cv_rgb)
    return pil_image

# =============================================================================
# 5. MAIN FUNCTION: SHOW ORIGINAL + MULTIPLE PREPROCESSED VERSIONS
# =============================================================================
def display_random_image_with_different_preprocessing():
    """
    - Loads a random image from the dataset.
    - Parses YOLO bounding boxes.
    - Shows four images side by side:
        (1) Original (with BBoxes)
        (2) White-Balanced (with BBoxes)
        (3) CLAHE-Enhanced (with BBoxes)
        (4) CLAHE + Gamma Correction (with BBoxes)
    - Displays the filename as well.
    """
    if not image_files or not annotation_files:
        print("No images or annotations found!")
        return

    # Select a random image
    random_image_path = random.choice(image_files)
    annotation_path = labels_path / (random_image_path.stem + ".txt")

    # If no annotation found, skip
    if not annotation_path.exists():
        print(f"No annotation file found for {random_image_path.name}")
        return

    # Load image with PIL
    with Image.open(random_image_path) as img:
        # Get size for YOLO coordinate parsing
        w, h = img.size

        # Get bounding boxes
        bboxes = parse_yolo_annotations(annotation_path, w, h)

        # -----------------------------
        # # 1) Original with BBoxes
        # # -----------------------------
        # original_with_bboxes = draw_bounding_boxes(img, bboxes, color="red")
        #
        # # -----------------------------
        # # 2) White-Balanced
        # # -----------------------------
        # cv_bgr = pil_to_bgr(img)
        # wb_bgr = white_balance_bgr(cv_bgr)
        # wb_pil = bgr_to_pil(wb_bgr)
        # wb_with_bboxes = draw_bounding_boxes(wb_pil, bboxes, color="blue")

        # -----------------------------
        # 3) CLAHE-Enhanced
        # -----------------------------
        clahe_bgr = enhance_clahe_bgr(cv_bgr, clip_limit=2.0, grid_size=(8,8))
        clahe_pil = bgr_to_pil(clahe_bgr)
        clahe_with_bboxes = draw_bounding_boxes(clahe_pil, bboxes, color="green")

        # -----------------------------
        # 4) CLAHE + Gamma Correction
        # -----------------------------
        clahe_gamma_bgr = gamma_correction_bgr(clahe_bgr, gamma=1.2)
        clahe_gamma_pil = bgr_to_pil(clahe_gamma_bgr)
        clahe_gamma_with_bboxes = draw_bounding_boxes(clahe_gamma_pil, bboxes, color="orange")

        # Display side by side
        fig, axes = plt.subplots(1, 4, figsize=(24, 6))
        fig.suptitle(f"File: {random_image_path.name}", fontsize=16, y=1.05)

        # # Original
        # axes[0].imshow(original_with_bboxes)
        # axes[0].set_title("Original")
        # axes[0].axis("off")
        #
        # # White Balanced
        # axes[1].imshow(wb_with_bboxes)
        # axes[1].set_title("White-Balanced")
        # axes[1].axis("off")
        #
        # # CLAHE Enhanced
        # axes[2].imshow(clahe_with_bboxes)
        # axes[2].set_title("CLAHE-Enhanced")
        # axes[2].axis("off")

        # CLAHE + Gamma
        axes[3].imshow(clahe_gamma_with_bboxes)
        axes[3].set_title("CLAHE + Gamma")
        axes[3].axis("off")

        plt.tight_layout()
        plt.show()

# =============================================================================
# 6. EXECUTION
# =============================================================================
display_random_image_with_different_preprocessing()


NameError: name 'cv_bgr' is not defined